<a href="https://colab.research.google.com/github/Jane-wondering/HelloWorld/blob/master/Introduction_to_NLP_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import spacy
import nltk
import gensim
import gensim.corpora as corpora
import re
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def clean(docs):
  data = docs.content.values.tolist()
  #remove emails, new lines, and single quotes
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
  data = [re.sub('\s+', ' ', sent) for sent in data]
  data = [re.sub("\'", "", sent) for sent in data]
  #lower case, tokenize, remove punct
  data = [[word for word in gensim.utils.simple_preprocess(str(doc), deacc=True) if word not in stop_words] for doc in data]
  for sent in data:
    sent = ([token.lemma_ for token in sent if token in ['NOUN', 'ADJ', 'VERB', 'ADV']])
  return data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

In [0]:
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [0]:
df.iloc[4,:]['content']

'From: jcm@head-cfa.harvard.edu (Jonathan McDowell)\nSubject: Re: Shuttle Launch Question\nOrganization: Smithsonian Astrophysical Observatory, Cambridge, MA,  USA\nDistribution: sci\nLines: 23\n\nFrom article <C5owCB.n3p@world.std.com>, by tombaker@world.std.com (Tom A Baker):\n>>In article <C5JLwx.4H9.1@cs.cmu.edu>, ETRAT@ttacs1.ttu.edu (Pack Rat) writes...\n>>>"Clear caution & warning memory.  Verify no unexpected\n>>>errors. ...".  I am wondering what an "expected error" might\n>>>be.  Sorry if this is a really dumb question, but\n> \n> Parity errors in memory or previously known conditions that were waivered.\n>    "Yes that is an error, but we already knew about it"\n> I\'d be curious as to what the real meaning of the quote is.\n> \n> tom\n\n\nMy understanding is that the \'expected errors\' are basically\nknown bugs in the warning system software - things are checked\nthat don\'t have the right values in yet because they aren\'t\nset till after launch, and suchlike. Rather than

In [0]:
cleaned_data = clean(df)

In [0]:
id2word = corpora.Dictionary(cleaned_data)
corpus = [id2word.doc2bow(text) for text in cleaned_data]

In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=10, 
                                            random_state=0, 
                                            update_every=1, 
                                            chunksize=100, 
                                            alpha='auto', 
                                            per_word_topics=True)

In [0]:
lda_model.top_topics(corpus)

In [0]:
model = gensim.models.Word2Vec(cleaned_data)

In [0]:
similar = model.most_similar('dream')
similar

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('tears', 0.9466832876205444),
 ('boys', 0.9422131776809692),
 ('riots', 0.9416725635528564),
 ('gross', 0.940330445766449),
 ('deserving', 0.9392300248146057),
 ('hurts', 0.9380338788032532),
 ('falls', 0.934821367263794),
 ('healthy', 0.9341471195220947),
 ('helpless', 0.9316052198410034),
 ('grew', 0.9299856424331665)]

In [0]:
dissimilar_words = model.doesnt_match('france spain banana orange china america'.split())
dissimilar_words

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'america'

In [0]:
!pip install -U sentence-transformers

In [0]:
from sentence_transformers import SentenceTransformer
from nltk.tokenize.treebank import TreebankWordDetokenizer
bert_data = []
for doc in cleaned_data:
  bert_data.append(TreebankWordDetokenizer().detokenize(doc))
doc_model = SentenceTransformer('bert-base-nli-mean-tokens')

doc_embeddings = doc_model.encode(bert_data)

100%|██████████| 405M/405M [00:53<00:00, 7.63MB/s]
